In [1]:
import json
import yaml
from mstrio.connection import Connection
from mstr_robotics.report import cube
from mstr_robotics.prepare_AI_data import parse_json
from mstr_robotics.mstr_classes import mstr_global
from mstr_robotics.json_checksum_handler import json_checksum_handler
from mstr_robotics.json_compare import JSONComparator,compare_mstr_objects
from mstr_robotics.read_out_prj_obj import read_gen
from mstr_robotics.redis_db import  redis_bi_analysis,redis_mstr_json
from mstr_robotics._helper import msic
from mstr_robotics._connectors import mstr_api
from mstr_robotics.prepare_AI_data import export_mstr_md,mstr_to_json
from mstr_robotics.redis_db import fetch_it_all
import pandas as pd


i_mstr_global=mstr_global()
i_msic=msic()
i_read_gen=read_gen()
i_parse_json=parse_json()
i_json_checksum_handler=json_checksum_handler()
i_mstr_to_json=mstr_to_json()
i_mstr_api=mstr_api()
i_redis_mstr_json=redis_mstr_json()
i_export_mstr_md=export_mstr_md()
i_compare_mstr_objects=compare_mstr_objects()
i_JSONComparator=JSONComparator()


with open('..\\config\\mstr_redis_y.yml', 'r') as openfile:
    mstr_redis_y = yaml.safe_load(openfile)

with open('..\\config\\user_d.json', 'r') as openfile:
    user_d = json.load(openfile)
conn_params =  user_d["conn_params"]

redis_con_d=mstr_redis_y["redis_env_d"]["mstr_dev"]
project_prefix=mstr_redis_y["project_prefix"]
prefix_map=mstr_redis_y["prefix_map"]
searches_used_in_prp_d_l=mstr_redis_y["searches_used_in_prp_d_l"]

## Connect to MSTR & Redis

In [2]:
conn = Connection(**conn_params)

i_redis_bi_analysis = redis_bi_analysis( 
    host=redis_con_d["host"],
    port=redis_con_d["port"],
    password=redis_con_d["password"],
    username=redis_con_d["username"],
    decode_responses=redis_con_d["decode_responses"]
)
org_project_id="B7CA92F04B9FAE8D941C3E9B7E0CD754"
comp_project_id="74F9B2164869627AF637FDA5D4A121B3"

cube_display_difference_id="70EDB2364AE577328CB5F283351051CE"
cube_display_difference_name="display_difference"
read_out_cbe_fold_id="9981CB214C79D75EC342A781539A4E5A"

org_prefix=project_prefix[org_project_id]
comp_prefix=project_prefix[comp_project_id]

Connection to Strategy One Intelligence Server has been established.
No project selected.


## Save objects to redis

### Daily Search Update

In [3]:
conn.select_project(comp_project_id)
search_result = i_mstr_api.run_mstr_search(conn=conn,
                    search_id="96648F2B492150A6AA27DDB3744E32B4")
print(search_result)

if search_result["totalItems"] > 0:
    all_obj_d_l=search_result["result"]

len(all_obj_d_l)

{'totalItems': 27, 'result': [{'name': 'Customer Detail report', 'id': '5D55D4504907E73F560BFBA95AD22A66', 'type': 3, 'subtype': 768, 'dateCreated': '2025-01-22T11:25:14.000+0000', 'dateModified': '2025-11-27T08:56:08.000+0000', 'version': '72003C9A48206FB747F4E99D6E99F0E2', 'acg': 255, 'owner': {'name': 'Administrator', 'id': '54F3D26011D2896560009A8E67019608', 'expired': False}, 'extType': 1, 'viewMedia': 134217728, 'certifiedInfo': {'certified': False}, 'projectId': '74F9B2164869627AF637FDA5D4A121B3', 'managed': False}, {'name': 'Cost', 'id': '7FD5B69611D5AC76C000D98A4CC5F24F', 'type': 4, 'subtype': 1024, 'dateCreated': '2001-09-18T20:56:27.000+0000', 'dateModified': '2025-11-27T08:54:16.000+0000', 'version': 'FA6D8D924CEA41B5ABC2EC97ADD53647', 'acg': 255, 'owner': {'name': 'Administrator', 'id': '54F3D26011D2896560009A8E67019608', 'expired': False}, 'extType': 0, 'projectId': '74F9B2164869627AF637FDA5D4A121B3', 'managed': False}, {'name': 'Detailed data', 'id': '98EB31B54122FFB738E

27

### Shortcut Folder

In [ ]:
folder_id="B2A671234A1B55E512EC3F83A24A150A"
conn.select_project(comp_project_id)
all_obj_d_l=i_mstr_global.get_obj_from_sh_fold(conn,folder_id=folder_id)

### Single Objects

In [ ]:
all_obj_d_l=[]
all_obj_d_l.append({"id":"B1D84B7D4F34D8E7B98F1185FF35C26C","type":"3","subtype":"768"})
all_obj_d_l.append({"id":"C060EF484E1352B26D945CBAF191F2B8","type":"3","subtype":"768"})

In [4]:
for pre in project_prefix:
    env_prefix=project_prefix[pre]
    conn.select_project(pre)


    err_d_l=i_redis_mstr_json.save_obj_json_to_redis(i_redis_bi_analysis=i_redis_bi_analysis                                                    
                                                        ,conn=conn
                                                        ,prefix_map=prefix_map
                                                        , all_obj_d_l=all_obj_d_l
                                                        , env_prefix=env_prefix)

err_d_l

Uploaded object definition for 5D55D4504907E73F560BFBA95AD22A66
Uploaded object definition for 7FD5B69611D5AC76C000D98A4CC5F24F
Uploaded object definition for 98EB31B54122FFB738E6E08A2F29421A
Uploaded object definition for 1C0A3D964E21D5D517D915958D284694
Uploaded object definition for 1C4BF0AE452247D0649F63B845862B19
Uploaded object definition for 4C05177011D3E877C000B3B2D86C964F
Uploaded object definition for 4C051DB611D3E877C000B3B2D86C964F
Uploaded object definition for 6F4FF0C84F271FC9851E9F8F502FF1C0
Uploaded object definition for 862780DC499A14D74FEC7EB2EF317DA2
Uploaded object definition for 8D67915D11D3E4981000E787EC6DE8A4
Uploaded object definition for 8D67917E11D3E4981000E787EC6DE8A4
Uploaded object definition for 8D67917F11D3E4981000E787EC6DE8A4
Uploaded object definition for 8D67927611D3E4981000E787EC6DE8A4
Uploaded object definition for 8D67933211D3E4981000E787EC6DE8A4
Uploaded object definition for 8D67936811D3E4981000E787EC6DE8A4
Uploaded object definition for 8D6793AA1

[]

In [5]:
json_diff_d_l=[]
for obj in all_obj_d_l:
    comp_d={}
    obj_id=obj["id"]
    conn.select_project(org_project_id)
    comp_d["org_j_f"]={"project_id":org_project_id,"object_id":obj_id,"redis_key":i_redis_mstr_json.bld_redis_key(conn,obj_id,org_prefix)}
    conn.select_project(comp_project_id)
    comp_d["comp_j_f"]={"project_id":comp_project_id,"object_id":obj_id,"redis_key":i_redis_mstr_json.bld_redis_key(conn,obj_id,comp_prefix)}
    json_diff_d_l.append(comp_d.copy())
org_root_object_id_l=[]
comp_root_object_id_l=[]

recursive_fg=True

for obj in json_diff_d_l:
    #print(obj.keys())
    org_root_object_id_l.append(obj["org_j_f"]["redis_key"])
    comp_root_object_id_l.append(obj["comp_j_f"]["redis_key"])

i_fetch_it_all=fetch_it_all(i_redis_bi_analysis)
all_org_objects_df = pd.DataFrame(i_fetch_it_all.fetch_all_objects_recursively(root_object_l=org_root_object_id_l,recursive_fg=recursive_fg))
all_comp_objects_df = pd.DataFrame(i_fetch_it_all.fetch_all_objects_recursively(root_object_l=comp_root_object_id_l,recursive_fg=recursive_fg))


In [6]:
diff_d_l=i_compare_mstr_objects.compare_objects( all_org_objects_df, all_comp_objects_df)

i_cube=cube()
conn.select_project("74F9B2164869627AF637FDA5D4A121B3")

df=pd.DataFrame(diff_d_l)
df["no_number_path"] = df["path_list"].apply(i_JSONComparator.remove_bracket_numbers)
for col in df.columns:
    df[col] = df[col].astype('string')

dict_list = df.to_dict('records')
df_1=pd.DataFrame(dict_list)

obj_prp_rel_d_l=[{"df":df_1,"tbl_name":"obj_prp_rel_df", "update_policy":"Replace" }]

i_cube.upload_cube_mult_table(conn=conn,tbl_upd_dict=obj_prp_rel_d_l,mtdi_id=cube_display_difference_id,
                              cube_name=cube_display_difference_name,folder_id=read_out_cbe_fold_id,force=True)



no subtype in {'id': 'BA38E83548881BF83570CCABE2CB9B4B', 'search_id': 'BA38E83548881BF83570CCABE2CB9B4B', 'obj_type': 39}
no subtype in {}
no subtype in {'id': 'E6A7C8EE4BB767E8380E3A896092C0E6', 'search_id': 'E6A7C8EE4BB767E8380E3A896092C0E6', 'obj_type': 39}
no subtype in {}
SuperCube object named: 'display_difference' with ID: '70EDB2364AE577328CB5F283351051CE'


Uploading 1/1:   0%|          | 0/1 [00:00<?, ?it/s]

Error uploading data to dataset 70EDB2364AE577328CB5F283351051CE
Could not decode the response from the I-Server. Please check if I-Server is running correctly


Uploading 1/1: 100%|██████████| 1/1 [00:02<00:00,  2.29s/it, rows=261]

Error publishing uploaded data for dataset with ID 70EDB2364AE577328CB5F283351051CE Cancelling publication.
Could not decode the response from the I-Server. Please check if I-Server is running correctly


Super cube 'display_difference' published successfully.


'70EDB2364AE577328CB5F283351051CE'